In [1]:
# !pip uninstall tokenziers==0.11.1
# !pip install tokenizers==0.9.4

In [48]:
# from transformers import GPT2LMHeadModel, GPT2TokenizerFast
from transformers import AutoTokenizer, T5ForConditionalGeneration
from soft_prompt_tuning import SoftEmbedding
import torch

In [49]:
# tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")
# model = GPT2LMHeadModel.from_pretrained('gpt2')
tokenizer = AutoTokenizer.from_pretrained("checkpoint/t5-base")
model = T5ForConditionalGeneration.from_pretrained("checkpoint/t5-base")

In [50]:
# model.get_input_embeddings()
n_tokens = 5
initialize_from_vocab = True
s_wte = SoftEmbedding(model.get_input_embeddings(),
                      n_tokens=n_tokens, 
                      initialize_from_vocab=initialize_from_vocab)
model.set_input_embeddings(s_wte)

In [45]:
input_query = "which are the directors of the films written by the writer of [The Green Mile]"

inputs = tokenizer(input_query, return_tensors="pt", max_length=512, truncation=True)

inputs['input_ids'] = torch.cat([torch.full((1,n_tokens), 50256), inputs['input_ids']], 1)
inputs['attention_mask'] = torch.cat([torch.full((1,n_tokens), 1), inputs['attention_mask']], 1)

print(inputs['input_ids'])
print(inputs['attention_mask'])

outputs = model.generate(
    inputs["input_ids"], max_length=21, min_length=15, length_penalty=2.0, num_beams=4, early_stopping=True
)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

tensor([[50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
            84,    33,     8, 10392,    13,     8,  4852,  1545,    57,     8,
          4346,    13,   784,   634,  1862, 11705,   908,     1]])
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

RuntimeError: The size of tensor a (200) must match the size of tensor b (101) at non-singleton dimension 3

In [8]:
# input_query = "which are the directors of the films written by the writer of [The Green Mile]"
input_query = "which are the directors"

inputs = tokenizer(input_query, return_tensors="pt", truncation=True)
inputs['input_ids'] = torch.cat([torch.full((1,n_tokens), 50256), inputs['input_ids']], 1)
inputs['attention_mask'] = torch.cat([torch.full((1,n_tokens), 1), inputs['attention_mask']], 1)

print(inputs['input_ids'].shape)
tokens_to_generate = 10

# outputs = model.generate(**inputs, max_length=inputs['input_ids'].size(1)+tokens_to_generate, use_cache=False)
# outputs = model.generate(**inputs, max_length=inputs['input_ids'].size(1)+tokens_to_generate, use_cache=False)
outputs = model.generate(**inputs, max_length=inputs['input_ids'].size(1)+tokens_to_generate, use_cache=False)

torch.Size([1, 25])


In [9]:
outputs = model.generate(
    inputs["input_ids"], max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True
)

RuntimeError: The size of tensor a (40) must match the size of tensor b (21) at non-singleton dimension 3

In [51]:
inputs = tokenizer("may the force", return_tensors="pt")

# need to pad attention_mask and input_ids to be full seq_len + n_learned_tokens
# even though it does not matter what you pad input_ids with, it's just to make HF happy
inputs['input_ids'] = torch.cat([torch.full((1,n_tokens), 50256), inputs['input_ids']], 1)
inputs['attention_mask'] = torch.cat([torch.full((1,n_tokens), 1), inputs['attention_mask']], 1)

tokens_to_generate = 10

outputs = model.generate(**inputs, max_length=inputs['input_ids'].size(1)+tokens_to_generate, use_cache=False)

In [52]:
print(outputs)
print(tokenizer.decode(outputs[0]))

tensor([[0, 1]])
<pad></s>
